In [28]:
try:
    if is_init:
        pass
except:
    %pwd
    %cd ..
    %pwd
    is_init = True

In [29]:
from pathlib import Path
from src.utils import FileUtils
from config import Config
from utils import rmse
import torch
import numpy as np
import pandas as pd
import altair as alt
from src.predictor import Predictor
import random
import itertools
data_folder = '../local_data'
work_folder = '../local_work'
configs_folder = '../configs'

In [30]:
inc_params = torch.load(
                Path(f"{data_folder}/inconel-onehot.pt"))["laser_params"]
steel_params = torch.load(
                Path(f"{data_folder}/steel-onehot.pt"))["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
max_speed, max_spacing = steel_params.max(0)[0][0].item(), steel_params.max(0)[0][1].item()
min_speed, min_spacing = steel_params.min(0)[0][0].item(), steel_params.min(0)[0][1].item()

In [4]:
# read_file
# read_file = pd.read_csv('../src/notebooks/sample_input.csv')
# desired_emiss = interpolate(read_file) #torch.from_numpy(np.array(torch.randn(1, 800) * 3))

include_inconel = False #Change ME
include_stainless = True #Change ME
round_laser_params = True #Change Me

def get_test_data(filepath):
    stain_data = torch.load(
                    Path(filepath))#["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
    wave_data = stain_data["interpolated_wavelength"]
    emiss_data = stain_data["interpolated_emissivity"]
    laser_params = stain_data["laser_params"]
    uids = stain_data["uids"]
    wave_test = wave_data[round(len(wave_data) * .9):]  
    emiss_test = emiss_data[round(len(wave_data) * .9):]
    laser_params = laser_params[round(len(wave_data) * .9):]
    return wave_test, emiss_test, laser_params, uids

In [10]:
wave_test, emiss_test, laser_params = get_test_data(f"{data_folder}/steel-onehot.pt")
# for emiss in emiss_test:
predictor = Predictor(emiss_test[0].reshape(1,800), include_inconel, include_stainless, round_laser_params = True)
y_hat_ss, y_hat_inc = predictor.run_inverse(predictor.desired)
#predictor.denormalize_decode_result()
    # direct_ss_emiss_y_hat, direct_inc_emiss_y_hat = predictor.run_direct(y_hat_ss, y_hat_inc)
    # best_substrate, best_params, best_rmse = predictor.best_predictor(True, True, direct_ss_emiss_y_hat, direct_inc_emiss_y_hat, predictor.desired)

/home/spencersong/metamaterials_ai/src


/home/spencersong/anaconda3/envs/meta/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/spencersong/anaconda3/envs/meta/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:217: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['direct_model.model.0.weight', 'direct_model.model.0.bias', 'direct_model.model.2.layers.0.block.0.weight', 'direct_model.model.2.layers.0.block.0.bias', 'direct_model.model.2.layers.1.block.0.weight', 'direct_model.model.2.layers.1.block.0.bias', 'direct_model.model.2.layers.2.block.0.weight', 'direct_model.model.2.layers.2.block.0.bias', 'direct_model.model.2.layers.3.block.0.weight', 'direct_model.model.2.layers.3.block.0.bias', 'direct_model.model.2.layers.4.block.0.weight', 'direct_model.model.2.

In [34]:
emiss_test.shape[0]

3038

In [11]:
y_hat_ss

tensor([[0.1677, 0.5185, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

In [7]:
# all_norm_params[0]

In [8]:
# i = 0
# found_index = -1
# all_norm_params  = torch.load(
#                 Path(f"{data_folder}/steel-onehot.pt"))["normalized_laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled

# for comb in all_norm_params:
#     i += 1
#     if torch.all(torch.eq(comb, y_hat_ss)):
#         found_index = i
# #y_final in steel_params

# found_index

In [25]:
wave_test, emiss_test, laser_params = get_test_data(f"{data_folder}/steel-onehot.pt")
steel_emiss = torch.load(Path(f"{data_folder}/steel-onehot.pt"))["emissivity"]
ss_to_minok = []
ss_in_data_same_params = []
ss_in_data_diff_params = []
watts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]
y_add = torch.empty(3, dtype=torch.float32)
for i in range(50):#len(emiss_test)):
    predictor = Predictor(emiss_test[i].reshape(1,800), include_inconel, include_stainless, round_laser_params = True)
    y_hat_ss, y_hat_inc = predictor.run_inverse(predictor.desired)
    #predictor.denormalize_decode_result()
        # direct_ss_emiss_y_hat, direct_inc_emiss_y_hat = predictor.run_direct(y_hat_ss, y_hat_inc)
        # best_substrate, best_params, best_rmse = predictor.best_predictor(True, True, direct_ss_emiss_y_hat, direct_inc_emiss_y_hat, predictor.desired)
    y_final = predictor.denorm_result(y_hat_ss, predictor.ss_max_speed, predictor.ss_max_spacing, predictor.ss_min_speed, predictor.ss_min_spacing) #reround laser parameters
    y_final, y_index = predictor.find_in_data(y_final) #get index of laser parameters matching in original data
    print(y_final)
    print(steel_params[y_index])
    if y_index == -1:
        watt_arg = torch.argmax(y_final[0][2:])
        y_add[0] = y_final[0][0]#(max_speed - min_speed) * (y_hat[0][0] + min_speed) # TODO call the scale
        y_add[1] = y_final[0][1]#(max_spacing - min_spacing) * (y_hat[0][1] + min_spacing) # TODO call the scale
        y_add[2]= watts[watt_arg]
        ss_to_minok.append(y_add)
    else:
    #     if torch.equal(y_final, steel_params[y_index]):
    #     #print(y_final)
    #     #print(steel_params[y_index])
    #     ss_in_data_same_params.append(steel_emiss[y_index])
    # else:
        ss_in_data_diff_params.append(steel_emiss[y_index])
print(len(ss_to_minok))
#print(len(ss_in_data_same_params))
print(len(ss_in_data_diff_params))


/home/spencersong/metamaterials_ai/src
tensor([[120.,  22.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,
           0.,   0.]])
tensor([110.,  36.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,
          0.,   0.])
/home/spencersong/metamaterials_ai/src
tensor([[100.,  40.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   1.]])
tensor([10., 13.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.])
/home/spencersong/metamaterials_ai/src
tensor([[370.,  41.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   1.]])
tensor([400.,  22.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.])
/home/spencersong/metamaterials_ai/src
tensor([[190.,  42.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   1.]])
tensor([150.,  31.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,
          0.,   0.])
/home/spencersong/metamaterials_ai/src
tensor(

In [27]:
ss_to_minok

[tensor([340.0000,  25.0000,   1.3000])]

In [16]:
y_index

-1

In [13]:
y_final, y_index

NameError: name 'y_index' is not defined

In [10]:
# stain_data = torch.load(
#                     Path(filepath))#["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
# wave_data = stain_data["interpolated_wavelength"]
# emiss_data = stain_data["interpolated_emissivity"]
# laser_params = stain_data["laser_params"]
# def check_params_in_data(filepath):
#     stain_data = torch.load(
#                     Path(filepath))#["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
#     wave_data = stain_data["interpolated_wavelength"]
#     emiss_data = stain_data["interpolated_emissivity"]
#     laser_params = stain_data["laser_params"]
    
#     return wave_test, emiss_test, laser_params
# laser_params
#torch.nonzero(y_final[0] == steel_params.sum(dim=1)==steel_params.size(1))
i = 0
found_index = -1
for comb in steel_params:
    i += 1
    if torch.all(torch.eq(comb, y_final)):
        found_index = i
#y_final in steel_params

found_index

19800

In [13]:
# inc_params = torch.load(
#                 Path(f"{data_folder}/inconel-onehot.pt"))["emissivity"]
steel_emiss = torch.load(Path(f"{data_folder}/stainless.pt"))#["emissivity"]

steel_emiss.keys()

dict_keys(['wavelength', 'laser_params', 'emissivity', 'uids', 'interpolated_emissivity', 'interpolated_wavelength', 'normalized_laser_params'])

In [16]:
steel_params["interpolated_emissivity"]

tensor([130.,  41.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          1.,   0.])

In [ ]:
torch.isin()y_final

In [ ]:

def is_in_data(self, rounded_y_hat, ):
        indices_have_in_data
        indices_not_have_in_data
        return rounded_y_hat, indices_have_in_data, indeces_not_have_in_data

# Inverse

In [ ]:
inverse_ss_filepath = Path(f"{work_folder}/saved_best/I-1-res-ann-stainless.ckpt") #CHANGEME
inverse_inc_filepath = Path(f"{work_folder}/saved_best/I-1-res-ann-inconel.ckpt") #CHANGEME
print(inverse_ss_filepath)
print(inverse_inc_filepath)
if not Path.is_file(inverse_ss_filepath):
    raise Exception(f'Model file does not exist at {inverse_ss_filepath}!')
if not Path.is_file(inverse_inc_filepath):
    raise Exception(f'Model file does not exist at {inverse_inc_filepath}!')

In [ ]:
def get_test_data(filepath):
    stain_data = torch.load(
                    Path(f"{data_folder}/{filepath}"))#["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
    wave_data = stain_data["interpolated_wavelength"]
    emiss_data = stain_data["interpolated_emissivity"]
    laser_params = stain_data["laser_params"]#.keys()#["wavelenght"]
    #print(wave_test.max(1), wave_test.min(1)) #2.5002, 11.9479
    #print(emiss_test.max(1)[0])#.max(1), emiss_test.min(1)) #0, 1
    wave_test = wave_data[round(len(wave_data) * .9):]  
    emiss_test = emiss_data[round(len(wave_data) * .9):]
    return wave_test, emiss_test, laser_params
#print(max_wave, min_wave)

In [ ]:
# pass test data into inverse model, get predicted parameters
from src.models import InverseModel
i_model = InverseModel.load_from_checkpoint(inverse_ss_filepath, strict=False)
i_model.eval()

wave_test, emiss_test, laser_params = get_test_data('stainless-steel-revised-shuffled.pt')

test_y_hats = []#torch.empty(size = len(emiss_test))
for emiss in emiss_test:
    with torch.no_grad():
        test_y_hats.append(i_model(emiss.reshape(1, 800)))

In [ ]:
laser_test = laser_params
max_speed, max_spacing = laser_test.max(0)[0][0].item(), laser_test.max(0)[0][1].item()
min_speed, min_spacing = laser_test.min(0)[0][0].item(), laser_test.min(0)[0][1].item()
laser_output = [denormalize_decode_result(x.reshape(1,14), max_speed, max_spacing, min_speed, min_spacing) for x in laser_test]

# denormalize_decode_result(laser_test[3].reshape(1,14),max_speed, max_spacing, min_speed, min_spacing)

# Direct Model

In [ ]:
from src.models import DirectModel
direct_filepath = Path(""
    f"{work_folder}/saved_best/D-1-res-ann-stainless.ckpt") #CHANGEME
print(direct_filepath)
if not Path.is_file(direct_filepath):
    raise Exception(f'Model file does not exist at {direct_filepath}!')

In [ ]:
d_model = DirectModel.load_from_checkpoint(direct_filepath)
d_model.eval()
emiss_y_hats = []
for x in test_y_hats:
    with torch.no_grad():
        emiss_y_hats.append(d_model(x.reshape(1,14)))

In [ ]:
#calculate rmse
pred_rmse_vals = []
columns = ["wavelength", "emissivity", "prediction", "example_number", "laser_params"]
viz_coords = pd.DataFrame(columns = columns)
for i in range(9):#len(emiss_test)):
    pred_rmse_vals.append(rmse(emiss_y_hats[i], emiss_test[i]).item())
    for j in range(len(wave_test[i].tolist())):
        viz_coords.append([wave_test[i].tolist()[j], emiss_test[i].tolist()[j], "True", i, laser_output[i].tolist()])
        viz_coords.append([wave_test[i].tolist()[j], emiss_y_hats[i].flatten().tolist()[j], "Prediction", i, laser_output[i].tolist()])
stdev = np.std(pred_rmse_vals)
print(f'Direct Filepath: {direct_filepath}')
print(f'Inverse Filepath: {inverse_ss_filepath}')
print(f'SD of RMSEs on test data: {stdev}')


In [ ]:
alt.Chart(viz_coords).mark_line().encode(
    x='wavelength',
    y='emissivity'
).facet(facet = 'example_number:N')#, header = 'laser_params:N')#.encode(text = 'laser_params:N').

In [ ]:
#visualize predictions vs original
